In [ ]:
#| default_exp base_model

In [ ]:
#| exporti 
from typing import *
from pydantic import BaseModel as PydanticBaseModel
from pydantic.generics import GenericModel as PydanticGenericModel
import json
from json2html import json2html
from IPython.display import HTML,JSON
import inspect
import yaml
from archetypon.delegates import delegates
import logging
from pandas import DataFrame as PandasDataFrame
from pydantic import validator
import pandas as pd

# Base Model
> Extending Pydantic's BaseModel with Jupyter-specific utilities

## Custom Pandas DataFrame
> Add classmethods to Pandas' DataFrame object to allow for Pydantic validation

In [ ]:
#|export 

class DataFrame(PandasDataFrame):
    """Subclassed from Pandas DataFrame. Includes classmethods used in Pydantic Validation"""
    
    @classmethod
    def __get_validators__(cls):
        yield cls.validate_dataframe
    
    @classmethod
    def __modify_schema__(cls, field_schema: Dict[str, Any]) -> None:
        pass
    
    @classmethod
    def validate_dataframe(cls,v):

        return PandasDataFrame(v)

## DBT Schema
> Support functions to convert Pydantic Model into a DBT Schema

In [ ]:

#|exporti

def dict_to_yaml(data: dict) -> str:
    # convert the dictionary to a yaml string
    yaml_str = yaml.dump(data,sort_keys = False)

    return yaml_str

In [ ]:

#|exporti

def pydantic_to_dbt(model: Type[PydanticBaseModel]) -> dict:
    # convert the model to a dictionary
    model_dict = model.schema()

    # create a dictionary for the dbt model
    dbt_model = {
        "version": 2,
        "name": model.__name__.lower(),  # use the model's class name as the table name
        "description":model_dict.get('description'),
        "columns": []
    }
    dbt_model = {k:v for k,v in dbt_model.items() if v or k=='columns'}

    # add the columns from the pydantic model to the dbt model
    for field_name, field in model_dict["properties"].items():

        column = {
            "name": field_name,
            "description":field.get('description'),
            "type": field["type"],
        }
        dbt_model["columns"].append({k:v for k,v in column.items() if v})
    return dbt_model


In [ ]:
#|exporti

class Base():

    @delegates(PydanticBaseModel.json)
    def display_json(
        self,
        json_loads_kwargs: dict = {}, # passed to json.loads()
        display_kwargs: dict = {}, # passed to IPython.display.JSON()
        **kwargs
    ): 
        """Helper function to display json in jupyter lab using kwargs passed to pydantic's .json() method"""
        return JSON(
            json.loads(self.json(**kwargs),**json_loads_kwargs),
            **display_kwargs
        )
    
    @delegates(PydanticBaseModel.json)
    def display_html(self,**kwargs):
        return HTML(
            json2html.convert(self.json(**kwargs))
        )
    
    @classmethod
    @delegates(PydanticBaseModel.schema_json)
    def display_schema_json(cls,**kwargs):
        """Helper function to display schema json in jupyter lab using kwargs passed to pydantic's .json() method"""
        return JSON(
            json.loads(cls.schema_json(**kwargs))
        )

    @classmethod
    @delegates(PydanticBaseModel.schema_json)
    def schema_html(cls,**kwargs):
        return HTML(
            json2html.convert(cls.schema_json(**kwargs))
        )

    @classmethod
    @delegates(PydanticBaseModel.schema)
    def schema_yml(cls,**kwargs):
        dbt = pydantic_to_dbt(cls)

        return dict_to_yaml(dbt)

    def _repr_html_(self):
        try:
            return self.display_html(**self.Display.html).data
        except Exception as e:
            logging.warning(e)
            pass

    def _repr_json_(self):
        try:
            return self.display_json(**self.Display.json).data
        except Exception as e:
            logging.warning(e)
            pass

    class Display:
        json = {}
        html = {}
    
    class Config:
        json_encoders = {
            PandasDataFrame: lambda df: json.loads(df.to_json(date_format='iso'))
        }


In [ ]:
#|export

class BaseModel(PydanticBaseModel,Base):
    """
    Custom implementation of Pydantic's Base Model.

    Includes `_repr_json_` and `_repr_html_` methods for nice displays in Jupyter Lab and Jupyter Notebook, respectively.

    """    
    class Config(Base.Config):
        pass

In [ ]:
import datetime as dt
from pydantic import validator
from dateutil.relativedelta import relativedelta
from pydantic import ValidationError

In [ ]:
class Person(BaseModel):
    name: str
    dob: dt.date
    age: Optional[int] 
    
    @validator('age',always=True)
    def _validate_age(cls,v,values):
        difference_in_years = relativedelta(
            dt.date.today(), 
            values['dob']
        ).years
        if v and v!=difference_in_years:
            raise ValueError("You're lying about your age!")
        return difference_in_years

In [ ]:
me = Person(
    name = 'Humble Chuck',
    dob = '1994-06-11'
)
me

name,Humble Chuck
dob,1994-06-11
age,28


In [ ]:
try:
    me = Person(
        name = 'Humble Chuck',
        dob = '1994-06-11',
        age = 27
    )
except ValidationError as e:
    print(e)

1 validation error for Person
age
  You're lying about your age! (type=value_error)


In [ ]:
#|export 

class GenericModel(PydanticGenericModel,Base):
    """
    Custom implementation of Pydantic's Generic Model.

    Includes `_repr_json_` and `_repr_html_` methods for nice displays in Jupyter Lab and Jupyter Notebook, respectively.

    """    

    class Config(Base.Config):
        pass

In [ ]:
class ModelWithDataFrame(BaseModel):
    df: DataFrame
    

In [ ]:
data = {
    'a':[1],
    'b':[2]
}
dataframe = DataFrame(data) 
model = ModelWithDataFrame(df=dataframe)
model

ModelWithDataFrame(df=   a  b
0  1  2)

In [ ]:
model

ModelWithDataFrame(df=   a  b
0  1  2)

## Generic DataFrame Model
> Anything passed to 'data' will be loaded as a dataframe.

In [ ]:
#|exporti

DataFrameT = TypeVar('DataFrameT')

In [ ]:
#| export

class DataFrameModel(GenericModel,Generic[DataFrameT]):
    """Generic DataFrame model. Anything passed to the 'data' attribute will be parsed as a DataFrame"""
    data: DataFrameT = None
    
    @validator('data',pre=True,always=True)
    def create_dataframe(cls,v):
        return pd.DataFrame(v)
    
    # @delegates(PydanticBaseModel.dict)
    # def to_df(self,**kwargs):
    #     """convert data to dataframe with **kwargs from Pydantics .dict() method"""
    #     return 
    
    def _repr_html_(self):
        
        df_html = self.data._repr_html_()
        schema = self.schema()
        html_fields = [
            f"<header><b>{schema_field}</b>: {schema[schema_field]}\n</header>"
            for schema_field in ['title','description']
        ]
        for field in self.__fields__.keys():
            if field!='data':
                html_fields.append(
                    f'<header><b>{field}</b>: {getattr(self,field)}</header>'
                )
        return ''.join(
            x for x in html_fields + ['<header><b>DataFrame</b>: </header>',df_html]
        )

    def _repr_json_(self):
        pass

In [ ]:
from pydantic import HttpUrl,Field
import pandas as pd

In [ ]:
class WFUVRecentlyPlayed(DataFrameModel):
    """A DataFrame of recently played songs from WFUV"""
    
    source:ClassVar[HttpUrl] = "https://wfuv.org/playlist"
    timestamp: dt.datetime = Field(description='The time that the data was collected',default_factory = dt.datetime.now)
    
    def __init__(self,*args,**kwargs):
        data = pd.read_html(self.source)[0]
        super().__init__(data=data,*args,**kwargs)
        

In [ ]:
recently_played = WFUVRecentlyPlayed()
recently_played

,Time,Song Title,Artist
0,"02/10, 1:38pm",The Greatest,Cat Power
1,"02/10, 1:31pm",Same Ol Mistakes,Rihanna
2,"02/10, 1:26pm",This Year,Emily King
3,"02/10, 1:23pm",Part of the Band,The 1975
4,"02/10, 1:19pm",Disarm,Smashing Pumpkins
...,...,...,...
345,"02/09, 1:16pm",Vacation,The Go-Go's
346,"02/09, 1:07pm",The Payback,James Brown
347,"02/09, 1:03pm",New Gold,Gorillaz
348,"02/09, 1:00pm",Modern Girl,Sleater-Kinney


In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()